# Database Setup

Data is originally from <br>
https://github.com/binlist<br>
https://github.com/iannuttall/binlist-data<br>
and manually entering bins. This data has been combined and modified.

Imports

In [2]:
# Generic Import
import pandas as pd
from pymongo import MongoClient
import numpy as np
import json

db_cert = "ca-certificate.crt" #PATH/TO/ca-certificate.crt

Connect to/create MongoDB database

In [3]:
def get_database(db_name):
    # Load connection string from creds.json
    with open('creds.json') as creds:
        CONNECTION_STRING = json.load(creds)['MongoDB']['connection_string'] + db_cert

    # Create a connection using MongoClient
    client = MongoClient(CONNECTION_STRING)

    return client[db_name] # return database

In [117]:
db = get_database('bin_list')
bins = db['bins'] # create collection in db bin_list

# Enter Values manually into the databse

See all bins currently in database

In [26]:
for bin in bins.find({}):
    #bins.delete_one(bin)
    print(bin['Bin'])

411810
409758
377935
545660
515557
403015
403446
414238
435880
484718
485246
494160
525362


Let's add BINS manually

In [5]:
response = input("Enter a BIN: ")
while response != "exit":
    if len(response) < 6:
        print("Please enter a 6 digit bin number.")
        response = input("Enter a BIN: ")
        continue
    if len([bin for bin in bins.find({'Bin': response})]) == 0:
        info = {'Bin': response}
        info['Type'] = input("Type of Card (Debit/Credit): ")
        info['Reloadable'] = input("Is the card Reloadable? (Y/N): ")
        info['Category'] = input("Category (Prepaid, Gift, Credit, Debit): ")
        info['Company'] = input("Company (Vanilla, MyVanilla, etc.): ")
        info['Distributor'] = input("Financial Distributor (InComm Financial Services): ")
        info['Issuer'] = input("Issuer (bank that issues): ")
        info['Website'] = input("Website (include https:// and trailing /): ")
        info['Customer Service'] = input('Support number (1-833-322-6760): ')
        info['Country'] = input("USA")
        print("Submitting: ")
        for k, v in info.items():
            print(f"{k}: {v}")
        bins.insert_one(info).inserted_id
        print("Saved")
        response = input("Enter a BIN: ")
    else:
        print("BIN Already Uploaded.")
        response = input("Enter a BIN: ")
print("Finished")

Finished


View number of bins in the database now

In [118]:
len([bin for bin in bins.find({})])

338734

# Adding CSV's to the database

Let's clean up the first CSV and then upload to the database

In [7]:
bin1 = pd.read_csv('binlist-data.csv')
bin1

,bin,brand,type,category,issuer,alpha_2,alpha_3,country,latitude,longitude,bank_phone,bank_url
0,19627,PRIVATE LABEL,DEBIT,NaN,NaN,US,USA,United States,37.0902,-95.71290,NaN,NaN
1,21502,PRIVATE LABEL,DEBIT,NaN,NaN,US,USA,United States,37.0902,-95.71290,NaN,NaN
2,42410,PRIVATE LABEL,DEBIT,NaN,NaN,US,USA,United States,37.0902,-95.71290,NaN,NaN
3,57164,PRIVATE LABEL,DEBIT,NaN,NaN,US,USA,United States,37.0902,-95.71290,NaN,NaN
4,63047,VISA,DEBIT,NaN,NaN,US,USA,United States,37.0902,-95.71290,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
343058,45719896,VISA,DEBIT,DANKORT,SPAR NORD,DK,DNK,Denmark,56.2639,9.50179,NaN,NaN
343059,45719897,VISA,DEBIT,DANKORT,SPAR NORD,DK,DNK,Denmark,56.2639,9.50179,NaN,NaN
343060,45719898,VISA,DEBIT,DANKORT,SPAR NORD,DK,DNK,Denmark,56.2639,9.50179,NaN,NaN
343061,45719899,VISA,DEBIT,DANKORT,BANKNORDIK,DK,DNK,Denmark,56.2639,9.50179,NaN,NaN


Let's drop unused columns from this CSV

In [39]:
# bin1 = bin1.drop(columns = ['alpha_2', 'country', 'latitude', 'longitude'])
bin1

,bin,brand,type,category,issuer,alpha_3,bank_phone,bank_url
0,19627,PRIVATE LABEL,DEBIT,NaN,NaN,USA,NaN,NaN
1,21502,PRIVATE LABEL,DEBIT,NaN,NaN,USA,NaN,NaN
2,42410,PRIVATE LABEL,DEBIT,NaN,NaN,USA,NaN,NaN
3,57164,PRIVATE LABEL,DEBIT,NaN,NaN,USA,NaN,NaN
4,63047,VISA,DEBIT,NaN,NaN,USA,NaN,NaN
...,...,...,...,...,...,...,...,...
343058,45719896,VISA,DEBIT,DANKORT,SPAR NORD,DNK,NaN,NaN
343059,45719897,VISA,DEBIT,DANKORT,SPAR NORD,DNK,NaN,NaN
343060,45719898,VISA,DEBIT,DANKORT,SPAR NORD,DNK,NaN,NaN
343061,45719899,VISA,DEBIT,DANKORT,BANKNORDIK,DNK,NaN,NaN


Create a for loop to add this information to the DB (we do NOT want to overwrite current data)

In [5]:
def add_bin(bin_info):
    if len([bin for bin in bins.find({'Bin': bin_info['bin']})]) == 0:
        info = {'Bin': bin_info['bin']}
        info['Type'] = bin_info['type']
        info['Reloadable'] = ''
        info['Category'] = bin_info['type']
        info['Company'] = ''
        info['Distributor'] = ''
        info['Issuer'] = bin_info['issuer']
        info['Website'] = bin_info['bank_url']
        info['Customer Service'] = bin_info['bank_phone']
        info['Country'] = bin_info['alpha_3']
        bins.insert_one(info).inserted_id
    else:
        print("BIN Already Uploaded.")

In [ ]:
bin1.apply(add_bin, axis = 1)

Now we must redo the process for the second CSV

In [6]:
bin2 = pd.read_csv('ranges.csv')
bin2

,iin_start,iin_end,number_length,number_luhn,scheme,brand,type,prepaid,country,bank_name,bank_logo,bank_url,bank_phone,bank_city
0,341142,NaN,NaN,NaN,amex,NaN,credit,NaN,US,AMERICAN EXPRESS,NaN,www.americanexpress.com,NaN,NaN
1,342562,NaN,NaN,NaN,amex,NaN,credit,NaN,US,AMERICAN EXPRESS,NaN,www.americanexpress.com,NaN,NaN
2,360218,NaN,NaN,NaN,diners,Diners Club International,credit,NaN,EC,DINERS CLUB DEL ECUADOR S.A.,NaN,www.dinersclub.com.ec,+593 2 297 3100,Quito
3,360324,NaN,NaN,NaN,diners,Diners Club International,credit,NaN,CO,Banco Davivienda S.A.,NaN,www.davivienda.com,+57 1 3383838,Bogota D.C.
4,361766,NaN,NaN,NaN,diners,Diners Club,credit,NaN,CZ,DINERS CLUB INTERNATIONAL CS,NaN,www.dinersclub.cz,+420 255 712 712,PRAGUE
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5800,622698,NaN,NaN,NaN,unionpay,NaN,debit,NaN,CN,CHINA CITIC BANK,NaN,bank.ecitic.com,+86 95558,NaN
5801,651621,NaN,NaN,NaN,discover,NaN,credit,NaN,US,DISCOVER,NaN,NaN,8003472683,NaN
5802,655066,NaN,NaN,NaN,discover,NaN,credit,NaN,US,DISCOVER,NaN,NaN,8003472683,NaN
5803,659524,NaN,NaN,NaN,discover,NaN,credit,NaN,US,DISCOVER,NaN,NaN,8003472683,NaN


In [7]:
bin2 = bin2.drop(columns = ['iin_end', 'number_length', 'number_luhn', 'scheme', 'bank_logo', 'bank_city'])
bin2.head(5)

,iin_start,brand,type,prepaid,country,bank_name,bank_url,bank_phone
0,341142,NaN,credit,NaN,US,AMERICAN EXPRESS,www.americanexpress.com,NaN
1,342562,NaN,credit,NaN,US,AMERICAN EXPRESS,www.americanexpress.com,NaN
2,360218,Diners Club International,credit,NaN,EC,DINERS CLUB DEL ECUADOR S.A.,www.dinersclub.com.ec,+593 2 297 3100
3,360324,Diners Club International,credit,NaN,CO,Banco Davivienda S.A.,www.davivienda.com,+57 1 3383838
4,361766,Diners Club,credit,NaN,CZ,DINERS CLUB INTERNATIONAL CS,www.dinersclub.cz,+420 255 712 712


In [8]:
def add_bin2(bin_info):
    if len([bin for bin in bins.find({'Bin': bin_info['iin_start']})]) == 0:
        info = {'Bin': bin_info['iin_start']}
        info['Type'] = bin_info['brand']
        info['Reloadable'] = ''
        info['Category'] = bin_info['prepaid']
        info['Company'] = ''
        info['Distributor'] = ''
        info['Issuer'] = bin_info['bank_name']
        info['Website'] = bin_info['bank_url']
        info['Customer Service'] = bin_info['bank_phone']
        info['Country'] = bin_info['country']
        bins.insert_one(info).inserted_id
    else:
        print("BIN Already Uploaded.")

In [ ]:
bin2.apply(add_bin2, axis = 1)

Change string bins to integers to be compatible with API

In [143]:
db = get_database('bin_list')
bins = db['bins'] # create collection in db bin_list
for bin in bins.find({'Bin': {'$type': 'string'}}):
    bins.update_one({'_id': bin['_id']}, {'$set': {'Bin': int(bin['Bin'])}})

Change NaN to empty strings

In [4]:
db = get_database('bin_list')
bins = db['bins'] # create collection in db bin_list
for column in ['Type', 'Reloadable', 'Category', 'Company', 'Distributor', 'Issuer', 'Website', 'Customer Service', 'Country']:
    for bin in bins.find({column: {'$eq': np.nan}}):
        bins.update_one({'_id': bin['_id']}, {'$set': {column: ''}})

NameError: name 'json' is not defined